In [1]:
# パッケージのimport
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [2]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cpu


次のセルは、2-2-3_Dataset_DataLoaderからコピペすれば良い。

In [4]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn

# ファイルパスのリストを取得
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)   #(BGRの平均)
input_size = 300

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train",
                           transform=DataTransform(input_size, color_mean),
                           transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val",
                           transform=DataTransform(input_size, color_mean),
                           transform_anno=Anno_xml2list(voc_classes))

batch_size = 32

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [5]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],      # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],      # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],         # DBoxの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBoxの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315], # DBoxの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2],],
}

# SSDネットワークモデル
net = SSD(phase='train', cfg=ssd_cfg)

# SSDの初期の重みを設定
# ssdのvgg部分に重みをロードする
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssdのその他のネットワークの重みはHeの初期値で初期化

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)
            
# Heの初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるかを確認
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('使用デバイス: ', device)

print('ネットワーク設定完了：学習済みの重みをロードしました')

使用デバイス:  cpu
ネットワーク設定完了：学習済みの重みをロードしました


In [6]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

In [8]:
# モデルを学習させる関数を作成

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    # GPUが使えるかを確認
    device = torch.device('duca:0' if torch.cuda.is_available() else 'cpu')
    print('使用デバイス：', device)
    
    # ネットワークをGPUへ
    net.to(device)
    
    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
    
    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0
    epoch_val_loss = 0.0
    logs = []
    
    # epochのループ
    for epoch in range(num_epochs+1):
            
            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()
            
            print('------------')
            print('Epoch {}/{}'.format(epoch+1, num_epochs))
            print('------------')
            
            # epochごとの訓練と検証のループ
            for phase in ['train', 'val']:
                if phase == 'train':
                    net.train()
                else:
                    if ((epoch+1)%10 == 0):
                        net.eval()
                        print('-------------')
                        print(' (val) ')
                    else:
                        # 検証は10回に1回岳行う
                        continue
                # データローダーからminibatchずつ取り出すループ
                for images, targets in dataloaders_dict[phase]:
                    
                    # GPUが使えるならGPUにデータを送る
                    images = images.to(device)
                    targets = [ann.to(device) for ann in targets]   # リストの各要素のテンソルをGPUへ
                    
                    # optimizerを初期化
                    optimizer.zero_grad()
                    
                    # 順伝搬(forward)計算
                    with torch.set_grad_enabled(phase == 'train'):
                        # 順伝搬(forward)計算
                        outputs = net(images)
                        
                        # 損失の計算
                        loss_l, loss_c = criterion(outputs, targets)
                        loss = loss_l + loss_c
                        
                        # 訓練時はバックプロぱゲーション
                        if phase == 'train':
                            loss.backward()  # 勾配の計算
                            
                            # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                            nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
                            
                            optimizer.step()  # パラメータ更新
                            
                            if iteration%10 == 0:
                                t_iter_finish = time.time()
                                duration = t_iter_finish - t_iter_start
                                print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(iteration, loss.item(), duration))
                                t_iter_start = time.time()
                            
                            epoch_train_loss += loss.item()
                            iteration += 1
                        
                        # 検証時
                        else:
                            epoch_val_loss += loss.item()
            # epochのphaseごとのlossと正解率
            t_epoch_finish = time.time()
            print('-------------')
            print('epoch {} || Epoch_TRAIN_Loss:{:.4f} || Epoch_VAL_Loss:{:.4f}'.format(epoch+1, epoch_train_loss, epoch_val_loss))
            print('timer: {:.4f} sec.'.format(t_epoch_finish-t_epoch_start))
            t_epoch_start = time.time()
            
            # ログを保存
            log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
            logs.append(log_epoch)
            df = pd.DataFrame(logs)
            df.to_csv('log_output.csv')
            
            epoch_train_loss = 0.0
            epoch_val_loss = 0.0
            
            # ネットワークを保存する
            if (epoch+1)%10==0:
                torch.save(net.state_dict(), 'weights/ssd300_' + str(epoch+1) + '.pth')

In [9]:
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cpu
------------
Epoch 1/2
------------
イテレーション 10 || Loss: 16.7868 || 10iter: 1236.3402 sec.
イテレーション 20 || Loss: 12.0620 || 10iter: 1294.9243 sec.
イテレーション 30 || Loss: 11.2808 || 10iter: 1181.5768 sec.
イテレーション 40 || Loss: 10.4993 || 10iter: 1060.5368 sec.
イテレーション 50 || Loss: 9.1630 || 10iter: 1045.0743 sec.
イテレーション 60 || Loss: 8.4397 || 10iter: 1201.9270 sec.
イテレーション 70 || Loss: 8.6284 || 10iter: 1175.5674 sec.
イテレーション 80 || Loss: 8.3559 || 10iter: 1130.3071 sec.
イテレーション 90 || Loss: 8.2356 || 10iter: 1159.9245 sec.
イテレーション 100 || Loss: 7.7133 || 10iter: 1082.0769 sec.
イテレーション 110 || Loss: 7.8405 || 10iter: 1060.3961 sec.
イテレーション 120 || Loss: 7.8749 || 10iter: 1067.8293 sec.
イテレーション 130 || Loss: 7.7835 || 10iter: 1063.5420 sec.
イテレーション 140 || Loss: 7.7073 || 10iter: 1074.7977 sec.
イテレーション 150 || Loss: 8.0081 || 10iter: 1080.5340 sec.
イテレーション 160 || Loss: 7.2913 || 10iter: 1078.1935 sec.
イテレーション 170 || Loss: 7.0958 || 10iter: 1060.9519 sec.
-------------
epoch 1 || Epoch_TRAIN_Lo